In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 128)')
parser.add_argument('--epochs', type=int, default=20, metavar='N',
                    help='number of epochs to train (default: 164)')
parser.add_argument('--lr', type=float, default=0.1, metavar='LR',
                    help='learning rate (default: 0.1)')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                    help='SGD momentum (default: 0.9)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')

# Solve error in call the argparse package in jupyter notebook
args = parser.parse_args(args=[])
args.cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

# Dataloader
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./MNIST_data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./MNIST_data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),          
    batch_size=args.batch_size, shuffle=True, num_workers=2)

#Define Network, we implement LeNet here
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=(5,5),stride=1, padding=0)
        # add conv1_1 and conv1_2 two 3x3 conv layer
        # to that convolution layer Deeper
        self.conv1_1 = nn.Conv2d(6, 9, kernel_size=(3,3),stride=1, padding=1)
        self.conv1_2 = nn.Conv2d(9, 12, kernel_size=(3,3),stride=1, padding=1)
        self.conv2 = nn.Conv2d(12, 16, kernel_size=(5,5),stride=1, padding=0)
        # add more neuron to make your net 2x wider
        self.fc1 = nn.Linear(16*4*4, 240)
        self.fc2 = nn.Linear(240, 168)
        self.fc3 = nn.Linear(168, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        # apply relu to conv1_1
        out = F.relu(self.conv1_1(out))
        # apply relu to conv1_2
        out = F.relu(self.conv1_2(out))
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1) #flatten
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

model = Net()
if args.cuda:
	device = torch.device('cuda')
	model.to(device)
#define optimizer/loss function
Loss = nn.CrossEntropyLoss()

# try Optimizer Adam/RMSprop
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
#optimizer = optim.Adam(model.parameters(), lr=args.lr)
#optimizer = optim.RMSprop(model.parameters(), lr=args.lr, momentum=args.momentum)

#learning rate scheduling
def adjust_learning_rate(optimizer, epoch):
    if epoch < 10:
       lr = 0.01
    elif epoch < 15:
       lr = 0.001
    else: 
       lr = 0.0001

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#training function
def train(epoch):
    model.train()
    adjust_learning_rate(optimizer, epoch)

    for batch_idx, (data, target) in enumerate(train_loader):
        if args.cuda:
            data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = Loss(output, target)
        loss.backward()
        optimizer.step()
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))
        
        
#Testing function
def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        if args.cuda:
            data, target = data.to(device), target.to(device)
            with torch.no_grad():
                output = model(data)
        test_loss += Loss(output, target).data
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
#run and save model
for epoch in range(1, args.epochs + 1):
    train(epoch)
    test(epoch)
    savefilename = 'LeNet_'+str(epoch)+'.tar'
    torch.save({
            'epoch': epoch,
            'state_dict': model.state_dict(),
        }, savefilename)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300012
Train Epoch: 1 [128/60000 (0%)]	Loss: 2.304054
Train Epoch: 1 [256/60000 (0%)]	Loss: 2.300994
Train Epoch: 1 [384/60000 (1%)]	Loss: 2.297802
Train Epoch: 1 [512/60000 (1%)]	Loss: 2.304611
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.301169
Train Epoch: 1 [768/60000 (1%)]	Loss: 2.304244
Train Epoch: 1 [896/60000 (1%)]	Loss: 2.299167
Train Epoch: 1 [1024/60000 (2%)]	Loss: 2.304559
Train Epoch: 1 [1152/60000 (2%)]	Loss: 2.299467
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.299232
Train Epoch: 1 [1408/60000 (2%)]	Loss: 2.303802
Train Epoch: 1 [1536/60000 (3%)]	Loss: 2.307327
Train Epoch: 1 [1664/60000 (3%)]	Loss: 2.304097
Train Epoch: 1 [1792/60000 (3%)]	Loss: 2.301008
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.300938
Train Epoch: 1 [2048/60000 (3%)]	Loss: 2.299248
Train Epoch: 1 [2176/60000 (4%)]	Loss: 2.307272
Train Epoch: 1 [2304/60000 (4%)]	Loss: 2.295916
Train Epoch: 1 [2432/60000 (4%)]	Loss: 2.296552
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.300190
Tr